In [15]:
import requests
import time 
import pandas as pd
import bs4
from bs4 import BeautifulSoup

In [ ]:
URL = "https://www.indeed.com/jobs?q=data+scientist%2420%2C000&l=New+York&start=10"

try:
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.title.text
    print(title)
except:
    print("Connection refused by the server..")
    print("Let me sleep for 2 seconds")
    time.sleep(2)
    print("Continue...")

In [ ]:
soup.prettify()

In [ ]:
with open('page.txt', 'wb') as f:
    f.write(response.content)

In [ ]:
with open('page.txt', 'rb') as f:
    print (f.read())

In [ ]:
with open('page.html', 'wb') as f:
    f.write(response.content)

In [ ]:
def extract_job_title(soup):
    jobs = []

    for spans in soup.select('h2.jobTitle span'):
        
        try:
            job = spans['title']
            jobs.append(job)
#             print(jobs) 
        except:
            pass
            #print('fail')
    
    return jobs

jobs = extract_job_title(soup)
jobs

In [ ]:
def extract_companyName(soup):
    
    companyName = []
    comps  = soup.findAll(name = 'span', attrs={'class':'companyName'})
    for comp in comps: 
        companyName.append(comp.text)
        
    return companyName

companyNames = extract_companyName(soup)
companyNames

In [ ]:
def extract_companyLocation(soup):
    
    companyLocation = []
    Locs  = soup.findAll(name = 'div', attrs={'class':'companyLocation'})
    for loc in Locs: 
        companyLocation.append(loc.text)
        
    return companyLocation

locations = extract_companyLocation(soup)
locations

In [ ]:
def extract_jobSalaries(soup):

    salaries = []
    for svgs in soup.findAll(name = 'div', attrs={'class':'heading6'}):
        try:
            salary = svgs.svg['aria-label']
            
            if salary.startswith("Estimated"):
                salary = salary.split(' ',1)[1]
                salaries.append(salary)
            else:
                salary = svgs.text.split('F',1)[0]
                salaries.append(salary)
        except:
            pass
        
    return salaries

salaries = extract_jobSalaries(soup)
salaries

In [ ]:
def extract_Summaries(soup):

    Summaries = []
    for S in soup.findAll(name = 'div', attrs={'class':'job-snippet'}):
        summary = S.text.strip()
        Summaries.append(summary)
    return Summaries

salaries = extract_Summaries(soup)
salaries[1]

## Mearge All Togethor

In [21]:
max_results_per_city = 100
city_list=['cairo']#,'Maadi']#,'Nasr%20City','Giza','Banha','Toukh','Qalyub']
job_list =['data+scientist']#,data%20scientist','data%20science','data%20analyst','data%20analysis','data%20entry','data%20engineer','database','call%20center%20arabic']

In [22]:
columns = ['city','job_field','job_title', 'compny_name','location', 'summary', 'salary','URL']
sample_df = pd.DataFrame(columns = columns)
sample_df

,city,job_field,job_title,compny_name,location,summary,salary,URL


In [23]:
start_time = time.time()

for city in city_list:
    time_per_city = time.time()
    for job_field in job_list:
        job_per_city = time.time()
        #for start in range(0,max_results_per_city,15):
        URL='http://eg.indeed.com/jobs?q='+str(job_field)+'&l=' + str(city)#+'&start='+str(start)
        page = requests.get(URL)
        time.sleep(1)  #ensuring at least 1 second between page grabs
        soup = BeautifulSoup(page.text, "html.parser")

        print(URL,'\n')#,page.content)
        with open('{} in {}.txt'.format(job_field,city), 'a') as f:
            f.write('\n'+URL+'\n')

        #with open('{} in {}.txt'.format(job_field,city), 'ab') as f:
        #    f.write(page.content)

        # All posted jobs in this field
        for job in soup.findAll(name = 'div', attrs={'class':'job_seen_beacon'}):
            job_per_city = time.time()
            #specifying row num for index of job posting in dataframe
            num = (len(sample_df) + 1) 
            #creating an empty list to hold the data for each posting job
            job_post = [] 
            #append city name 
            job_post.append(city) 
            job_post.append(job_field) 

            #grabbing job title
            for spans in job.select('h2.jobTitle span'):
                try:
                    job_title = spans['title']
                    #print(job_title)
                    job_post.append(job_title)
                except:
                    print('job title fail')

            #grabbing company name
            for comp in job.findAll(name = 'span', attrs={'class':'companyName'}):
                job_post.append(comp.text)

            #grabbing location name
            for loc in job.findAll(name = 'div', attrs={'class':'companyLocation'}): 
                job_post.append(loc.text)

            #grabbing summary text
            for S in job.findAll(name = 'div', attrs={'class':'job-snippet'}):
                job_post.append(S.text.strip())

            #grabbing salary
            for svgs in job.findAll(name = 'div', attrs={'class':'heading6'}):
                try:
                    salary = svgs.svg['aria-label']

                    if salary.startswith("Estimated"):
                        salary = salary.split(' ',1)[1]
                        job_post.append(salary)
                    else:
                        salary = svgs.text.split('F',1)[0]
                        job_post.append(salary)
                except:
                    #pass
                    job_post.append('Not_found')
                    break
                    #print('salary fail')

            #appending list of job post info to dataframe at index num
            job_post.append(URL)
            print(job_post)
            sample_df.loc[num] = job_post

        print('finished {} in {} at {:.2f} sec'.format(job_field, city, (time.time() - job_per_city)))

    print('finish all {} jobs at {:.2f} sec'.format(city ,(time.time() - time_per_city)))

print ('total time: {:.2f} min'.format((time.time() - start_time)/60))

http://eg.indeed.com/jobs?q=data+scientist&l=cairo 

finished data+scientist in cairo at 1.69 sec
finish all cairo jobs at 1.70 sec
total time: 0.03 min


In [ ]:
sample_df['job_title'].unique()#.groupby(['city']).count()

In [ ]:
#saving sample_df as a local csv file — define your own local path to save contents 

sample_df.to_csv('myJobs.csv', encoding='utf-8')